1. 파인 튜닝을 위한 데이터 확보
- 1_web_crawl.ipynb에서 얻었던 token.txt를 KoGPT2의 fine tuning을 해 학습시켜보았다. 그리고 이를 불러와 학습에 활용해보았다.


아직 실행 안했고 코드만 쳐둠

In [4]:
#라이브러리 import
import gluonnlp as nlp #이거 아직 설치 안함pip
from gluonnlp.data import SentencepieceTokenizer
from transformers import TFGPT2LMHeadModel
import Tensorflow as tf
import numpy as np
import os

ModuleNotFoundError: No module named 'gluonnlp'

In [ ]:
#모델 생성 방식 따라 디렉토리 경로 입력하는 클래스
class GPT2Model(tf.keras.Model):
    def __init__(self, dir_path):
        super(GPT2Model, self).__init__()
        self.gpt2 = TFGPT2LMHeadModel.from_pretrained(dir_path)

    def call(self, inputs):
        return self.gpt2(inputs)[0]

In [2]:
#파라미터 불러오는 명령어
!wget https://www.dropbox.com/s/nzfa9xpzm4edp6o/gpt_ckpt.zip -O gpt_ckpt.zip
!unzip -o gpt_ckpt.zip

'wget'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
'unzip'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [3]:
BASE_MODEL_PATH = './gpt_ckpt'
gpt_model = GPT2Model(BASE_MODEL_PATH)

NameError: name 'GPT2Model' is not defined

In [ ]:
#토크나이저
BATCH_SIZE = 16
NUM_EPOCHS = 10
MAX_LEN = 30
TOKENIZER_PATH = '.gpt_ckpt/gpt2_kor_tokenizer.spiece'

tokenizer = SentencepieceTokenizer(TOKENIZER_PATH)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(TOKENIZER_PATH, mask_token=None, spe_token=None, cls_token=None, unknown_token='<unk>', padding_token='<pad>', bos_token='<s>', eos_token='</s>')

In [ ]:
#토크나이저 & 사전 학습 모델 통해 문장 만들기 함수
def tf_top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-99999):
    _logits = logits.numpy()
    top_k = min(top_k, logits.shape[-1])
    if top_k > 0:
        indicees_to_remove = logits < tf.math.top_k(logits, top_k)[0][...,-1,None]
        _logits[indicees_to_remove] = filter_value
    
    if top_p > 0.0:
        sorted_logits = tf.sort(logits, direction='DESCENDING')
        sorted_indices = tf.argsort(logits, direction='DESCENDING')
        cumulative_probs = tf.math.cunsum(tf.nn.softmax(sorted_logits, axis=-1), axis =-1)

        sorted_indices_to_remove = cumulative_probs > top_p
        sorted_indices_to_remove = tf.concat([[False], sorted_indices_to_remove[..., :-1]], axis=0)
        indicees_to_remove = sorted_indices[sorted_indices_to_remove].numpy().tolist()

        _logits[indicees_to_remove] = filter_value
    return tf.constant([_logits])

def generate_sent(seed_word, model, max_step=100, greedy=False, top_k=0, top_p=0.):
    sent = seed_word
    toked = tokenizer(sent)

    for _ in range(max_step):
        input_ids = tf.constant([vocab[vocab.bos_token],] + vocab[toked])[None, :]
        outputs = model(input_ids)[:, -1, :]
        if greedy:
            gen = vocab.to_tokens(tf.argmax(outputs, axis=-1).numpy().tolist()[0])
        else:
            output_logit = tf_top_k_top_p_filtering(outputs[0], top_k=top_k, top_p=top_p)
            gen = vocab.to_tokens(tf.random.categorical(output_logit, 1).numpy().tolist()[0])[0]
        if gen == '</s>':
            break
        sent += gen.replace('_', ' ')
        toked = tokenizer(sent)
    
    return sent


In [ ]:
#가사 집어넣기
DATA_IN_PATH = './data_in/KOR' #이 부분 바꿔야 할 가능성 아주 농후함, 파일 주소라서 없어도 될지도?
TRAIN_DATA_FILE = 'data.txt'

sents = [s[:-1] for s in open(DATA_IN_PATH + TRAIN_DATA_FILE).readlines()]

In [ ]:
#토크나이저에 넣기
input_data = []
output_data = []

for s in sents:
    tokens = [vocab[vocab.bos_token],] + vocab[tokenizer(s)] + [vocab[vocab.eos_token],]
    input_data.append(tokens[:-1])
    output_data.append(tokens[1:])

In [ ]:
#입출력 구성
input_data = pad_sequences(input_data, MAX_LEN, value=vocab[vocab.padding_token])
output_data = pad_sequences(output_data, MAX_LEN, value=vocab[vocab.padding_token])

input_data = np.array(input_data, dtype=np.int64)
output_data = np.array(output_data, dtype=np.int64)

In [ ]:
#손실함수 & 정확도 측정 설정
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

def accuracy_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    mask = tf.expand_dims(tf.cast(mask, dtype=pred.dtype), axis=1)
    pred *= mask
    acc = train_accuracy(real, pred)

    return tf.reduce_mean(acc)

In [ ]:
#모델 컴파일
gpt_model.compile(loss=loss_function, optimizer=tf.keras.optimizers.Adam(1e-4), metrics=[accuracy_function])

In [ ]:
#학습 실행
history = gpt_model.fit(input_data, output_data, batch_size = BATCH_SIZE, epochs = NUM_EPOCHS, validation_split = 0.1)

In [ ]:
#저장
DATA_OUT_PATH = './data_out'
model_name = 'tf2_gpt2_finetuned_model'
save_path = os.path.join(DATA_OUT_PATH, model_name)

if not os.path.exists(save_path):
    os.makedirs(save_path)

gpt_model.gpt2.save_pretrained(save_path)

loaded_gpt_model = GPT2Model(save_path)

In [ ]:
#결과 확인
print(generate_sent('어떠한', gpt_model, greedy=True))
print('-'*16)
print(generate_sent('어떠한', gpt_model, greedy=False))